In [23]:
import pandas as pd
import geopandas as gpd

## Load the length of the roads

In [24]:
df_roads = pd.read_csv("routes_2016_ofs.csv")
df_roads.head()

,Id,Total,Routes nationales,Routes cantonales,Routes communales
0,22,7711,205,2131,5375
1,23,4205,111,1860,2234
2,25,1357,27,266,1064
3,2,11915,209,2086,9620
4,10,3444,84,636,2724


## Load the number of cars registered in each canton

In [25]:
df_cars = pd.read_csv("px-x-1103020100_105.csv", sep=";", encoding='latin-1')
df_cars['id'] = df_cars.index+1

In [26]:
df_cars.sort_values("2017", ascending=False).head()

,Canton,2017,id
0,Zürich,737247,1
1,Bern / Berne,531863,2
21,Vaud,414628,22
18,Aargau,390489,19
16,St. Gallen,278804,17


## Load the cantons

In [27]:
df_cantons = pd.read_csv("cantons.csv")
df_cantons['id'] = df_cantons.index+1

In [28]:
df_cantons.head()

,Abr.,Canton,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),Nombre de communes,Langues officielles,id
0,ZH,Zurich,1351,Zurich,1 487 969,"18,3",1 729,"4,2","860,6",166,allemand,1
1,BE,Berne,1353,Berne,1 026 513,"12,6","5 959,44","14,4","172,2",347,"allemand,\nfrançais",2
2,LU,Lucerne,1332,Lucerne,403 397,5,"1 493,44","3,6","270,1",83,allemand,3
3,UR,Uri,1291,Altdorf,36 145,"0,4","1 076,57","2,6","33,6",20,allemand,4
4,SZ,Schwytz,1291,Schwytz,155 863,"1,9","906,92","2,2","171,9",30,allemand,5


## Load the number of roundabouts for each commune

In [29]:
gdf = gpd.read_file("count.geojson")

### Add a new column that shows the number of inhabitants per roundabout

In [30]:
gdf["COUNT_POND"] = gdf["EINWOHNERZ"] / gdf["COUNT"]

### Show the communes sorted by number of roundabouts

In [31]:
(gdf[["NAME", "COUNT", "EINWOHNERZ", "KANTONSNUM"]]
 .sort_values("COUNT", ascending=False).head())

,NAME,COUNT,EINWOHNERZ,KANTONSNUM
173,Sion,45,33879.0,23.0
608,Lausanne,37,135629.0,22.0
840,Bulle,34,21991.0,10.0
762,Thun,33,43500.0,2.0
431,Fribourg,30,38489.0,10.0


### Show the communes sorted by number of inhabitants per roundabouts

In [32]:
(gdf[["NAME", "COUNT", "COUNT_POND", "EINWOHNERZ", "KANTONSNUM"]]
 .sort_values("COUNT_POND", ascending=True).head())

,NAME,COUNT,COUNT_POND,EINWOHNERZ,KANTONSNUM
1201,Chavannes-le-Veyron,1,120.000000,120.0,22.0
1391,Jaberg,2,126.000000,252.0,2.0
796,Beurnevsin,1,126.000000,126.0,26.0
1206,Allaman,3,131.666667,395.0,22.0
1033,Montagny-prs-Yverdon,5,142.600000,713.0,22.0


## Create a new dataframe containing the total of roundabouts for each canton

In [33]:
cantons_count = pd.DataFrame(gdf.groupby("KANTONSNUM")["COUNT"].sum().sort_values(ascending=False))

### Merge the cantons data with the roundabouts count

In [34]:
cantons_count_merged = cantons_count.merge(df_cantons, left_index=True, right_on="id")
cantons_count_merged.head()

,COUNT,Abr.,Canton,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),Nombre de communes,Langues officielles,id
21,455,VD,Vaud,1803,Lausanne,784 822,"9,6","3 212,03","7,8","244,3",309,français,22
1,396,BE,Berne,1353,Berne,1 026 513,"12,6","5 959,44","14,4","172,2",347,"allemand,\nfrançais",2
0,304,ZH,Zurich,1351,Zurich,1 487 969,"18,3",1 729,"4,2","860,6",166,allemand,1
9,267,FR,Fribourg,1481,Fribourg,311 914,"3,8","1 670,7",4,"186,7",136,"français,\nallemand",10
22,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",126,"français,\nallemand",23


### Merge the cantons data with the cars count

In [35]:
cantons_count_merged = cantons_count_merged.merge(df_cars, left_on='id', right_on="id")
cantons_count_merged.head()

,COUNT,Abr.,Canton_x,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),Nombre de communes,Langues officielles,id,Canton_y,2017
0,455,VD,Vaud,1803,Lausanne,784 822,"9,6","3 212,03","7,8","244,3",309,français,22,Vaud,414628
1,396,BE,Berne,1353,Berne,1 026 513,"12,6","5 959,44","14,4","172,2",347,"allemand,\nfrançais",2,Bern / Berne,531863
2,304,ZH,Zurich,1351,Zurich,1 487 969,"18,3",1 729,"4,2","860,6",166,allemand,1,Zürich,737247
3,267,FR,Fribourg,1481,Fribourg,311 914,"3,8","1 670,7",4,"186,7",136,"français,\nallemand",10,Fribourg / Freiburg,183742
4,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",126,"français,\nallemand",23,Valais / Wallis,217251


### Merge the cantons data with the roads length

In [36]:
cantons_count_merged = cantons_count_merged.merge(df_roads, left_on='id', right_on="Id")
cantons_count_merged.head()

,COUNT,Abr.,Canton_x,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),Nombre de communes,Langues officielles,id,Canton_y,2017,Id,Total,Routes nationales,Routes cantonales,Routes communales
0,455,VD,Vaud,1803,Lausanne,784 822,"9,6","3 212,03","7,8","244,3",309,français,22,Vaud,414628,22,7711,205,2131,5375
1,396,BE,Berne,1353,Berne,1 026 513,"12,6","5 959,44","14,4","172,2",347,"allemand,\nfrançais",2,Bern / Berne,531863,2,11915,209,2086,9620
2,304,ZH,Zurich,1351,Zurich,1 487 969,"18,3",1 729,"4,2","860,6",166,allemand,1,Zürich,737247,1,7364,151,1620,5593
3,267,FR,Fribourg,1481,Fribourg,311 914,"3,8","1 670,7",4,"186,7",136,"français,\nallemand",10,Fribourg / Freiburg,183742,10,3444,84,636,2724
4,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",126,"français,\nallemand",23,Valais / Wallis,217251,23,4205,111,1860,2234


### Add new columns that shows the number of roundabouts related to the population

In [37]:
cantons_count_merged["COUNT_POND"] = cantons_count_merged["COUNT"] / pd.to_numeric(cantons_count_merged["Population\n(décembre 2016)"].str.replace(" ", ""))
cantons_count_merged["COUNT_POND_INV"] = pd.to_numeric(cantons_count_merged["Population\n(décembre 2016)"].str.replace(" ", ""))/cantons_count_merged["COUNT"]

In [38]:
cantons_count_merged.sort_values("COUNT_POND_INV", ascending=True)

,COUNT,Abr.,Canton_x,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),...,id,Canton_y,2017,Id,Total,Routes nationales,Routes cantonales,Routes communales,COUNT_POND,COUNT_POND_INV
12,82,JU,Jura,1979,Delémont,73 122,"0,9","838,55",2,"87,2",...,26,Jura,43083,26,1681,47,449,1185,0.001121,891.731707
3,267,FR,Fribourg,1481,Fribourg,311 914,"3,8","1 670,7",4,"186,7",...,10,Fribourg / Freiburg,183742,10,3444,84,636,2724,0.000856,1168.217228
4,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",...,23,Valais / Wallis,217251,23,4205,111,1860,2234,0.000734,1362.152610
11,110,NE,Neuchâtel,1815,Neuchâtel,178 567,"2,2","802,93","1,9","222,4",...,24,Neuchâtel,97241,24,1884,40,449,1395,0.000616,1623.336364
0,455,VD,Vaud,1803,Lausanne,784 822,"9,6","3 212,03","7,8","244,3",...,22,Vaud,414628,22,7711,205,2131,5375,0.000580,1724.883516
7,174,TI,Tessin,1803,Bellinzone,354 375,"4,4","2 812,2","6,8",126,...,21,Ticino,225337,21,3155,139,1059,1957,0.000491,2036.637931
9,129,TG,Thurgovie,1803,Frauenfeld,270 709,"3,3","991,02","2,4","273,2",...,20,Thurgau,170529,20,3186,43,802,2341,0.000477,2098.519380
6,224,GE,Genève,1815,Genève,495 325,"6,1","282,48","0,7","1 753,5",...,25,Genève,221714,25,1357,27,266,1064,0.000452,2211.272321
20,17,NW,Nidwald,1291,Stans,42 556,"0,5","275,9","0,7","154,2",...,7,Nidwalden,26656,7,240,26,74,140,0.000399,2503.294118
22,14,UR,Uri,1291,Altdorf,36 145,"0,4","1 076,57","2,6","33,6",...,4,Uri,19819,4,371,70,152,149,0.000387,2581.785714


In [39]:
cantons_count_merged.sort_values("COUNT", ascending=False)

,COUNT,Abr.,Canton_x,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),...,id,Canton_y,2017,Id,Total,Routes nationales,Routes cantonales,Routes communales,COUNT_POND,COUNT_POND_INV
0,455,VD,Vaud,1803,Lausanne,784 822,"9,6","3 212,03","7,8","244,3",...,22,Vaud,414628,22,7711,205,2131,5375,0.000580,1724.883516
1,396,BE,Berne,1353,Berne,1 026 513,"12,6","5 959,44","14,4","172,2",...,2,Bern / Berne,531863,2,11915,209,2086,9620,0.000386,2592.204545
2,304,ZH,Zurich,1351,Zurich,1 487 969,"18,3",1 729,"4,2","860,6",...,1,Zürich,737247,1,7364,151,1620,5593,0.000204,4894.634868
3,267,FR,Fribourg,1481,Fribourg,311 914,"3,8","1 670,7",4,"186,7",...,10,Fribourg / Freiburg,183742,10,3444,84,636,2724,0.000856,1168.217228
4,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",...,23,Valais / Wallis,217251,23,4205,111,1860,2234,0.000734,1362.152610
5,244,AG,Argovie,1803,Aarau,662 224,"8,1","1 403,73","3,4","471,8",...,19,Aargau,390489,19,5596,99,1173,4324,0.000368,2714.032787
6,224,GE,Genève,1815,Genève,495 325,"6,1","282,48","0,7","1 753,5",...,25,Genève,221714,25,1357,27,266,1064,0.000452,2211.272321
7,174,TI,Tessin,1803,Bellinzone,354 375,"4,4","2 812,2","6,8",126,...,21,Ticino,225337,21,3155,139,1059,1957,0.000491,2036.637931
8,136,LU,Lucerne,1332,Lucerne,403 397,5,"1 493,44","3,6","270,1",...,3,Luzern,216708,3,3240,59,522,2659,0.000337,2966.154412
9,129,TG,Thurgovie,1803,Frauenfeld,270 709,"3,3","991,02","2,4","273,2",...,20,Thurgau,170529,20,3186,43,802,2341,0.000477,2098.519380


### Add new columns that shows the number of roundabouts related to the population

In [138]:
cantons_count_merged["COUNT_POND_CARS"] = cantons_count_merged["2017"]/cantons_count_merged["COUNT"]

In [141]:
cantons_count_merged["COUNT_CARS_INHAB"] = cantons_count_merged["2017"]/pd.to_numeric(cantons_count_merged["Population\n(décembre 2016)"].str.replace(" ", ""))

In [142]:
cantons_count_merged.sort_values("COUNT_POND_CARS", ascending=True).head()

,COUNT,Abr.,Canton_x,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),Nombre de communes,Langues officielles,id,COUNT_POND,COUNT_POND_INV,Canton_y,2017,COUNT_POND_CARS,COUNT_CARS_INHAB
12,82,JU,Jura,1979,Delémont,73 122,"0,9","838,55",2,"87,2",55,"français,\nallemand",26,0.001121,891.731707,Jura,43083,525.402439,0.589193
3,267,FR,Fribourg,1481,Fribourg,311 914,"3,8","1 670,7",4,"186,7",136,"français,\nallemand",10,0.000856,1168.217228,Fribourg / Freiburg,183742,688.172285,0.589079
4,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",126,"français,\nallemand",23,0.000734,1362.152610,Valais / Wallis,217251,872.493976,0.640526
11,110,NE,Neuchâtel,1815,Neuchâtel,178 567,"2,2","802,93","1,9","222,4",31,français,24,0.000616,1623.336364,Neuchâtel,97241,884.009091,0.544563
0,455,VD,Vaud,1803,Lausanne,784 822,"9,6","3 212,03","7,8","244,3",309,français,22,0.000580,1724.883516,Vaud,414628,911.270330,0.528308


In [147]:
cantons_count_merged.sort_values("COUNT_CARS_INHAB", ascending=False)

,COUNT,Abr.,Canton_x,Depuis,Chef-lieu,Population (décembre 2016),Population (en % du total suisse),Superficie (en km²),Superficie (en % du total suisse),Densité (enhab./km²),Nombre de communes,Langues officielles,id,COUNT_POND,COUNT_POND_INV,Canton_y,2017,COUNT_POND_CARS,COUNT_CARS_INHAB
17,38,ZG,Zoug,1352,Zoug,123 948,"1,5","238,69","0,6","519,3",11,allemand,9,0.000307,3261.789474,Zug,80904,2129.052632,0.652725
16,39,SZ,Schwytz,1291,Schwytz,155 863,"1,9","906,92","2,2","171,9",30,allemand,5,0.000250,3996.487179,Schwyz,100054,2565.487179,0.641936
4,249,VS,Valais,1815,Sion,339 176,"4,2","5 224,25","12,7","64,9",126,"français,\nallemand",23,0.000734,1362.152610,Valais / Wallis,217251,872.493976,0.640526
7,174,TI,Tessin,1803,Bellinzone,354 375,"4,4","2 812,2","6,8",126,115,italien,21,0.000491,2036.637931,Ticino,225337,1295.040230,0.635872
9,129,TG,Thurgovie,1803,Frauenfeld,270 709,"3,3","991,02","2,4","273,2",80,allemand,20,0.000477,2098.519380,Thurgau,170529,1321.930233,0.629935
20,17,NW,Nidwald,1291,Stans,42 556,"0,5","275,9","0,7","154,2",11,allemand,7,0.000399,2503.294118,Nidwalden,26656,1568.000000,0.626375
25,5,AI,Appenzell Rhodes-Intérieures,1513,Appenzell,16 003,"0,2","172,52","0,4","92,8",6,allemand,16,0.000312,3200.600000,Appenzell Innerrhoden,9602,1920.400000,0.600012
21,14,OW,Obwald,1291,Sarnen,37 378,"0,5","490,59","1,2","76,2",7,allemand,6,0.000375,2669.857143,Obwalden,22288,1592.000000,0.596287
5,244,AG,Argovie,1803,Aarau,662 224,"8,1","1 403,73","3,4","471,8",212,allemand,19,0.000368,2714.032787,Aargau,390489,1600.364754,0.589663
12,82,JU,Jura,1979,Delémont,73 122,"0,9","838,55",2,"87,2",55,"français,\nallemand",26,0.001121,891.731707,Jura,43083,525.402439,0.589193


## Number of roundabouts related to the roads length

In [49]:
df_cantons_count_simple = df_cantons[["Canton", "id"]]
df_cantons_count_simple.head()

,Canton,id
0,Zurich,1
1,Berne,2
2,Lucerne,3
3,Uri,4
4,Schwytz,5


#### Merge the roads length with the cantons

In [67]:
df_roundabouts_roads = df_cantons_count_simple.merge(df_roads, left_on="id", right_on="Id").drop('Id', axis=1)
df_roundabouts_roads.head()

,Canton,id,Total,Routes nationales,Routes cantonales,Routes communales
0,Zurich,1,7364,151,1620,5593
1,Berne,2,11915,209,2086,9620
2,Lucerne,3,3240,59,522,2659
3,Uri,4,371,70,152,149
4,Schwytz,5,898,50,218,630


#### Merge the roudabouts count with the cantons and roads

In [68]:
df_roundabouts_roads = df_roundabouts_roads.merge(cantons_count, left_on="id", right_index=True)
df_roundabouts_roads.head()

,Canton,id,Total,Routes nationales,Routes cantonales,Routes communales,COUNT
0,Zurich,1,7364,151,1620,5593,304
1,Berne,2,11915,209,2086,9620,396
2,Lucerne,3,3240,59,522,2659,136
3,Uri,4,371,70,152,149,14
4,Schwytz,5,898,50,218,630,39


#### Add a new column showing the number of roudabouts for each km of road

In [69]:
df_roundabouts_roads["1 roundabout per n km of road"] = df_roundabouts_roads["Total"] / df_roundabouts_roads["COUNT"]
df_roundabouts_roads["n roundabouts per 100 km"] = df_roundabouts_roads["COUNT"] / df_roundabouts_roads["Total"] * 100
df_roundabouts_roads.sort_values("1 roundabout per n km of road", ascending=True)

,Canton,id,Total,Routes nationales,Routes cantonales,Routes communales,COUNT,1 roundabout per n km of road,n roundabouts per 100 km
24,Genève,25,1357,27,266,1064,224,6.058036,16.507001
9,Fribourg,10,3444,84,636,2724,267,12.898876,7.752613
6,Nidwald,7,240,26,74,140,17,14.117647,7.083333
8,Zoug,9,556,18,138,400,38,14.631579,6.834532
22,Valais,23,4205,111,1860,2234,249,16.887550,5.921522
21,Vaud,22,7711,205,2131,5375,455,16.947253,5.900661
23,Neuchâtel,24,1884,40,449,1395,110,17.127273,5.838641
20,Tessin,21,3155,139,1059,1957,174,18.132184,5.515055
25,Jura,26,1681,47,449,1185,82,20.500000,4.878049
11,Bâle-Ville,12,376,10,305,61,17,22.117647,4.521277


In [72]:
df_roundabouts_roads["n roundabouts per 100 km"].mean()

4.7276783081893505

In [73]:
print("Total number of roundabouts:", cantons_count_merged["COUNT"].sum())

Total number of roundabouts: 3295


In [91]:
pd.to_numeric(cantons_count_merged["Population\n(décembre 2016)"].str.replace(" ", "")).sum()

8430516

In [92]:
8430516 / 3295

2558.5784522003037

In [97]:
cantons_fr = ['10', '22', '23', '24', '25', '26']

In [99]:
df_cantons_fr = cantons_count_merged[cantons_count_merged['id'].isin(cantons_fr)]

In [100]:
df_cantons_fr["COUNT"].sum()

1387

In [101]:
pd.to_numeric(df_cantons_fr["Population\n(décembre 2016)"].str.replace(" ", "")).sum()

2182926

In [102]:
2182926 / 1387

1573.8471521268925

In [103]:
1387/3295

0.42094081942336875

In [104]:
2182926/8430516

0.2589314817740693

In [146]:
# Total number of cars
cantons_count_merged["2017"].sum()

4570823